# Charts for bus service opportunities by tract and CalEnviroScreen / Density Scores

* Density and CalEnviroScreen both grouped as low/med/high
* Provide some charts to accompany maps
* Heatmaps to show distribution?
* Scatterplot of pop density vs service density

In [1]:
import pandas as pd

import make_tract_viz

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
make_tract_viz.create_all_viz(CHART_IMG_PATH = "./test_img/")

In [ ]:
df = make_tract_viz.import_processed_data()
TOTAL_CA_TRACTS = len(df)

## Opportunity Tracts

Tracts in the lower right quandrant, but pick out the most extreme cases.

Tracts with > 75th ptile for pop/job density & < 25th ptile for service density.

Especially the medium-need tracts, these would be the ones dragging down the average for the middle yellow bar.

In [ ]:
# Grab the ones with density above 75th percentile
# and in the lower 25th percentile for service density
p75 = df.popjobs_sq_mi.quantile(0.75)
p25 = df.arrivals_per_1k_pj.quantile(0.25)

df = df.assign(
    flag_me = df.apply(lambda x: (x.popjobs_sq_mi > p75) and 
                       (x.arrivals_per_1k_pj < p25), axis=1)
)

pct_oppor_tracts = round(len(df[df.flag_me==1])/TOTAL_CA_TRACTS, 3)
print(f"# opportunity tracts: {len(df[df.flag_me==1])}")
print(f"% opportunity tracts of total: {pct_oppor_tracts*100}%")

### List these cities where opportunity tracts fall

In [ ]:
display_cols = [
    "County", "City", "ZIP"
]

for county in ["Santa Clara", "Los Angeles", "Sacramento"]: 
    df2 = df[df.flag_me==1].reset_index(drop=True)
    df3 = df2.drop_duplicates(subset=["County", "City"])
    display(df3[df3.County==county][display_cols])
    print(df3[df3.County==county].City.unique())